<a href="https://colab.research.google.com/github/mannb986/admissions_data_deep_learning/blob/main/admissions_data_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Regression with Admissions Data

The goal of this project is to create a regression model that predicts the likelihood that a student applying to graduate school will be accpeted based on various application factors (such as test scores). 

By analysing the parameters in the graduate admissions dataset I will use TensorFlow & Keras to create a regression model that can evaluate the chances of an applicant being admitted. 

## Data Loading & Observing

In [1]:
 from google.colab import files


uploaded = files.upload()

Saving admissions_data.csv to admissions_data.csv


In [2]:
import pandas as pd

admissions_df = pd.read_csv('admissions_data.csv')

In [3]:
admissions_df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [8]:
admissions_df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


The above shows the various parameters the admission officers use to evaluate university applicants. There is information on 500 applicants from various universities. 

All columns are numerical so won't requires any one-hot encoding. 

My features for the model will include all columns except `Serial No.` and `Chance of Admit`. `Chance of Admit` will be my labels for the model. 

In [9]:
features = admissions_df.iloc[:, 1:-1]

In [10]:
features.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0


In [11]:
labels = admissions_df.iloc[:, -1]

In [12]:
labels.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64

## Data Processing

In [14]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.33, random_state = 42)

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 

numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns

ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

In [17]:
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)

In [18]:
features_train_scaled = pd.DataFrame(features_train_scaled, columns = features_train.columns)
features_test_scaled = pd.DataFrame(features_test_scaled, columns = features_test.columns)

In [19]:
features_test_scaled.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,1.570357,1.423757,0.784816,0.621518,-0.001613,1.612511,0.873589
1,-0.251845,0.112531,0.784816,1.123225,0.538819,0.768277,0.873589
2,-0.160734,-0.379179,-0.950586,-1.385311,-1.082478,-1.578691,-1.144703
3,-0.434065,0.276434,-0.082885,-0.381897,-0.542045,0.177314,-1.144703
4,0.841476,0.768144,-0.082885,0.119811,-0.542045,0.785162,0.873589


## Building the Model

In [20]:
from tensorflow.keras.models import Sequential

my_model = Sequential()

In [21]:
from tensorflow.keras.layers import InputLayer

#creating the input layer with the shape corresponding to the number of features
input = InputLayer(input_shape=(features.shape[1],))
my_model.add(input)

In [22]:
from tensorflow.keras.layers import Dense

#adding one hidden layer with 128 neurons
my_model.add(Dense(128, activation='relu'))

In [23]:
#adding output layer with a single output
my_model.add(Dense(1))

In [24]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


## Initializing the Optimizer & Compiling the Model

In [25]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.01)

my_model.compile(loss = 'mse', metrics = 'mae', optimizer=opt)

## Fit & Evaluate Model

In [26]:
my_model.fit(features_train_scaled, labels_train, epochs=40, batch_size=1, verbose=1)

Epoch 1/40
335/335 [==============================] - 1s 1ms/step - loss: 0.1428 - mae: 0.2541
Epoch 2/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0137 - mae: 0.0904
Epoch 3/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0094 - mae: 0.0751
Epoch 4/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0073 - mae: 0.0643
Epoch 5/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0066 - mae: 0.0639
Epoch 6/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0054 - mae: 0.0584
Epoch 7/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0048 - mae: 0.0533
Epoch 8/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0061 - mae: 0.0598
Epoch 9/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0073 - mae: 0.0632
Epoch 10/40
335/335 [==============================] - 0s 1ms/step - loss: 0.0079 - mae: 0.0617
Epoch 11/40
335/335 [============================

In [27]:
res_mse, rse_mae = my_model.evaluate(features_test_scaled, labels_test, verbose=0)

In [28]:
print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  0.004267274402081966
MAE  0.044717706739902496


The results show a very good performance of the model. I will tune the model to understand the effects of tuning the hyperparameters. 

## Tuning Model

In [29]:
##Changing batch size from 1 to 4
my_model.fit(features_train_scaled, labels_train, epochs=40, batch_size=4, verbose=1)

Epoch 1/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0038 - mae: 0.0448
Epoch 2/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0038 - mae: 0.0445
Epoch 3/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0038 - mae: 0.0446
Epoch 4/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0036 - mae: 0.0433
Epoch 5/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.0446
Epoch 6/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.0436
Epoch 7/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.0441
Epoch 8/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.0437
Epoch 9/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.0439
Epoch 10/40
84/84 [==============================] - 0s 2ms/step - loss: 0.0037 - mae: 0.0444
Epoch 11/40
84/84 [==============================] - 0s 2ms/step - lo

In [30]:
res_mse, rse_mae = my_model.evaluate(features_test_scaled, labels_test, verbose=0)

print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  0.003916493151336908
MAE  0.04282800480723381


Comparing the results with where the batch size was 1, it can seen there is very little difference in the results. 

Let's try adding `EarlyStopping()` to increase efficiency. I will also add another hiddent layer to the model. 

In [32]:
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=40) 

my_model.fit(features_train_scaled, labels_train, epochs=40, batch_size=4, verbose=1, validation_split=0.2, callbacks=[stop])

Epoch 1/40
67/67 [==============================] - 0s 6ms/step - loss: 0.0044 - mae: 0.0508 - val_loss: 0.0022 - val_mae: 0.0347
Epoch 2/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0043 - mae: 0.0494 - val_loss: 0.0029 - val_mae: 0.0425
Epoch 3/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0045 - mae: 0.0492 - val_loss: 0.0028 - val_mae: 0.0438
Epoch 4/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0043 - mae: 0.0484 - val_loss: 0.0021 - val_mae: 0.0348
Epoch 5/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0043 - mae: 0.0501 - val_loss: 0.0021 - val_mae: 0.0335
Epoch 6/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0042 - mae: 0.0468 - val_loss: 0.0022 - val_mae: 0.0339
Epoch 7/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0044 - mae: 0.0475 - val_loss: 0.0024 - val_mae: 0.0358
Epoch 8/40
67/67 [==============================] - 0s 3ms/step - loss: 0.0042 - mae: 0.04

In [33]:
res_mse, rse_mae = my_model.evaluate(features_test_scaled, labels_test, verbose=0)

print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  0.003841502359136939
MAE  0.045660316944122314


In [35]:
##adding another hidden layer to the model.
my_model_2 = Sequential()
input_2 = InputLayer(input_shape=(features.shape[1],))
my_model_2.add(input_2)
my_model_2.add(Dense(128, activation='relu'))
my_model_2.add(Dense(64, activation='relu'))
my_model_2.add(Dense(1))
my_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 9,345
Trainable params: 9,345
Non-trainable params: 0
_________________________________________________________________


In [36]:
my_model_2.compile(loss = 'mse', metrics = 'mae', optimizer=opt)

In [37]:
my_model_2.fit(features_train_scaled, labels_train, epochs=40, batch_size=4, verbose=1, validation_split=0.2, callbacks=[stop])

Epoch 1/40
67/67 [==============================] - 1s 5ms/step - loss: 0.4933 - mae: 0.4971 - val_loss: 0.0059 - val_mae: 0.0568
Epoch 2/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0070 - mae: 0.0646 - val_loss: 0.0054 - val_mae: 0.0588
Epoch 3/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0062 - mae: 0.0583 - val_loss: 0.0049 - val_mae: 0.0534
Epoch 4/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0061 - mae: 0.0595 - val_loss: 0.0049 - val_mae: 0.0521
Epoch 5/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0057 - mae: 0.0573 - val_loss: 0.0059 - val_mae: 0.0621
Epoch 6/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0047 - mae: 0.0522 - val_loss: 0.0045 - val_mae: 0.0488
Epoch 7/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0058 - mae: 0.0565 - val_loss: 0.0039 - val_mae: 0.0456
Epoch 8/40
67/67 [==============================] - 0s 2ms/step - loss: 0.0039 - mae: 0.04

In [38]:
res_mse, rse_mae = my_model_2.evaluate(features_test_scaled, labels_test, verbose=0)

print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  0.0054812245070934296
MAE  0.057330161333084106


By adding a leyer there has been a slight decrease in performance. 